In [2]:
import icu

In [3]:
def transliterator_from_rules(name, rules):
    fromrules = icu.Transliterator.createFromRules(name, rules)
    icu.Transliterator.registerInstance(fromrules)
    return icu.Transliterator.createInstance(name)

In [13]:
irishlc_rules = """
:: NFD;
$uvowel=[AEIOU];
$wb=[^[:L:][:M:]];
$wb { ([nt]) } $uvowel → $1 '-';
:: lower;
:: NFC;
"""

In [14]:
irishlc = transliterator_from_rules('irishlc', irishlc_rules)

In [8]:
ulster_stress_rules = """
$wb=[^[:L:][:M:]];
$cons = [bcdfghjklmnpqrstvwxyz];
$vowel = [aeiouáéíóú];
$bvowel = [aouáóú{ae}];
$svowel = [eiéí];
:: irishlc;
$wb { anois } $wb → an'ˈ'ois;
$wb { arís } $wb → air'ˈ'ís;
$wb { isteach } $wb → ist'ˈ'each;
$wb { amach } $wb → am'ˈ'ach;
$wb $cons* { ($vowel) → 'ˈ'$1;
"""

In [9]:
stress = transliterator_from_rules('ulster_stress', ulster_stress_rules)

In [12]:
stress.transliterate("amach")

'amˈach'

In [57]:
ulster_g2p_rules = """
$wb=[^[:L:][:M:]];
$cons = [bcdfghjklmnpqrstvwxyz];
$vowel = [aeiouáéíóú];
$bvowel = [aouáóú{ae}];
$svowel = [eiéí];
:: irishlc;
:: ulster_stress;
$wb { l } ˈ? $bvowel → ʟˠ;
$wb { l } ˈ? $svowel → ʟʲ;
$wb { r → ɾˠ;
$bvowel $cons* { t → tˠ;
t } $cons* ˈ? $bvowel → tˠ;
$svowel $cons* { t → tʲ;
t } $cons* ˈ? $svowel → tʲ;
$bvowel $cons* { s → sˠ;
s } $cons* ˈ? $bvowel → sˠ;
$svowel $cons* { s → ʃ;
s } $cons* ˈ? $svowel → ʃ;
ˈeá → aː;
ˈá → aː;
a → ə ;
'ˈ' → 'ˈ';
"""

In [58]:
ulster_g2p = transliterator_from_rules('ulster_g2p', ulster_g2p_rules)

In [59]:
ulster_g2p.transliterate("anoit")

'ˈənoitʲ'